In [1]:
import csv
import xml.etree.ElementTree as ET
import json
import csv
import pyodbc
from geopy.geocoders import Nominatim
import requests
from datetime import datetime

In [2]:
# Load participant age, status, and type dictionaries
with open('dict_partecipant_age.json') as f1:
    dict_partecipant_age = json.load(f1)

with open('dict_partecipant_status.json') as f2:
    dict_partecipant_status = json.load(f2)

with open('dict_partecipant_type.json') as f3:
    dict_partecipant_type = json.load(f3)

In [3]:
# Function to compute additional date-related data
def compute_date_data(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    date = date_obj.date()
    day = date_obj.day
    month = date_obj.month
    year = date_obj.year
    quarter = (date_obj.month - 1) // 3 + 1
    day_of_week = date_obj.strftime('%A')
    return date, day, month, year, quarter, day_of_week

In [4]:
# Function to parse dates.xml and create a mapping of date_fk to real date
def parse_dates_xml(xml_file):
    date_mapping = {}
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for row in root.findall('.//row'):
        date = row.find('date').text
        date_pk = int(row.find('date_pk').text)
        date_mapping[date_pk] = date

    return date_mapping


In [5]:
# Function to parse dates.xml and create a mapping of date_fk to real date
def parse_dates_xml(xml_file):
    date_mapping = {}
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for row in root.findall('.//row'):
        date = row.find('date').text
        date_pk = int(row.find('date_pk').text)
        date_mapping[date_pk] = date

    return date_mapping


In [6]:
# Function to compute crime gravity using provided dictionaries
def compute_crime_gravity(x):
    gravity = dict_partecipant_age.get(x['participant_age_group'], 1) * \
              dict_partecipant_type.get(x['participant_type'], 1) * \
              dict_partecipant_status.get(x['participant_status'], 1)
    return gravity

In [7]:
def get_continent_by_country_code(country_code):
    if country_code:
        try:
            response = requests.get(f'https://restcountries.com/v3/alpha/{country_code.lower()}')
            data = response.json()
            continent = data[0]['region']
            return continent
        except Exception as e:
            print(f"Error fetching continent information: {e}")
    
    return None

In [8]:
def get_location_info(latitude, longitude):
    geolocator = Nominatim(user_agent="Lab_DSS_Group_ID_200")
    location = geolocator.reverse((latitude, longitude), language='en')

    if location is not None:
        address = location.address
        city = location.raw.get('address', {}).get('city') or location.raw.get('address', {}).get('town') or location.raw.get('address', {}).get('village') or location.raw.get('address', {}).get('county', None)

        state = location.raw.get('address', {}).get('state', None)
        #country = location.raw.get('address', {}).get('country', None) (Not calculated as the country is USA for all records, and the Continent is North America)

        #continent = get_continent_by_country_code(location.raw.get('address', {}).get('country_code', None)) 

        return {
            'city': city,
            'state': state,
            #'country': country,
            #'continent': continent
        }
    else:
        return None


In [9]:
# Function to insert data in batches
def insert_in_batches(conn, cursor, insert_query, data, batch_size):
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        cursor.executemany(insert_query, batch)
        conn.commit()

In [10]:
#Write to my db
# Connection string
server = 'tcp:lds.di.unipi.it'
username = 'Group_ID_200'
password = '89VIG10K'
database = 'Group_ID_200_DB'
connectionString = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password

In [11]:
# Connect to the SQL Server database
conn = pyodbc.connect(connectionString)
cursor = conn.cursor()

In [12]:
import csv

# ... (other imports and functions)

def insert_data_with_ID(conn, cursor, id_dict, table_name, key_dict, data_list, batch_size):
    key_tuple = tuple(key_dict.values())
    if key_tuple not in id_dict:
        id_dict[key_tuple] = next(iter(key_dict.values()))
        columns = ', '.join(key_dict.keys())
        placeholders = ', '.join(['?'] * len(key_dict))
        insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
        data_list.append(tuple(key_dict.values()))

        if len(data_list) >= batch_size:
            cursor.executemany(insert_query, data_list)
            conn.commit()
            data_list.clear()

def insert_data_without_ID(conn, cursor, table_name, key_dict, data_list, id_dict, id_list, batch_size):
    columns = ', '.join(key_dict.keys())
    placeholders = ', '.join(['?'] * len(key_dict))
    insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    select_query = 'SELECT SCOPE_IDENTITY();'

    key_tuple = tuple(key_dict.values())

    if key_tuple not in id_dict:
        data_list.append(tuple(key_dict.values()))

        if len(data_list) >= batch_size:
            cursor.executemany(insert_query, data_list)
            conn.commit()
            
            # Retrieve the IDs using SCOPE_IDENTITY()
            cursor.execute(select_query)
            id_values = cursor.fetchall()
            
            # Update the id_dict with the newly inserted IDs
            for idx, id_value in enumerate(id_values):
                id_dict[tuple(data_list[idx])] = id_value[0]

            data_list.clear()
    else:
        id_value = id_dict[key_tuple]
        id_list.append(id_value)


def split_and_integrate(csv_file):
    geo_id_dict = {}
    gun_id_dict = {}
    partecipant_id_dict = {}
    date_id_dict = {}
    incident_id_dict = {}
    custody_id_dict = {}
    # Parse dates.xml to get date mapping
    date_mapping = parse_dates_xml('dates.xml')

    # List of table names in your database
    table_names = ['Custody', 'Geography', 'Gun', 'Date', 'Incident', 'Partecipant']

    # Delete existing records in the tables
    for table_name in table_names:
        cursor.execute(f'DELETE FROM {table_name}')
        conn.commit()

    run_size = 10
    # Read and process Police.csv
    with open(csv_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        rows_to_upload = 170928 #sum(1 for row in reader)
        #next(reader)  # Skip the header row

        
        # Set the batch size
        if(rows_to_upload >= run_size):
            batch_size = run_size
        else:
            batch_size = rows_to_upload
        print(rows_to_upload, batch_size)
        # Counter to track the number of processed rows
        row_count = 0

        gun_data, participant_data, geo_data, date_data, incident_data, custody_data = [], [], [], [], [], []
        gun_id_dict, geo_id_dict, partecipant_id_dict = {}, {}, {}
        gun_id_list, geo_id_list, partecipant_id_list = [], [], []

        # ... (previous code)

        for row in reader:
            print("started working")
            custody_id, participant_age_group, participant_gender, participant_status, participant_type, latitude, longitude, gun_stolen, gun_type, incident_id, date_fk = row

            gun_stolen_bool = 1 if row["gun_stolen"] == 'Stolen' else 0
            gun_key_dict = {"is_stolen": gun_stolen_bool, "gun_type": row['gun_type']}
            gun_id = insert_data_without_ID(conn, cursor, 'Gun', gun_key_dict, gun_data, gun_id_dict, gun_id_list, batch_size)
            print("a")

            partecipant_key = {
                "age_group": row['participant_age_group'],
                "gender": row['participant_gender'],
                "type": row['participant_type'],
                "status": row['participant_status']
            }
            try:
                participant_id = insert_data_without_ID(conn, cursor, 'Partecipant', partecipant_key, participant_data, partecipant_id_dict, partecipant_id_list, batch_size)
            except Exception as e:
                print(f"Error inserting participant: {e}")
                print("Problematic row:", row)
                continue
            #participant_id = insert_data_without_ID(conn, cursor, 'Partecipant', partecipant_key, participant_data, partecipant_id_dict, partecipant_id_list, batch_size)
            print("b")

            latitude, longitude = float(row['latitude']), float(row['longitude'])
            location_info = get_location_info(latitude, longitude)
            city = location_info["city"]
            state = location_info["state"]
            country = "United States"
            continent = "North America"
            geo_key = {
                "latitude": str(latitude), "longitude": str(longitude), "city": city, "state": state, "country": country,
                "continent": continent
            }
            geo_id = insert_data_without_ID(conn, cursor, 'Geography', geo_key, geo_data, geo_id_dict, geo_id_list, batch_size)
            print("c")

            date_id = int(row['date_fk'])
            date_value = date_mapping[date_id]
            date, day, month, year, quarter, day_of_week = compute_date_data(date_value)
            date_key = {
                "date_id": date_id, "the_date": date, "the_day": day, "the_month": month, "the_year": year, "quarter": quarter,
                "day_of_week": day_of_week
            }
            insert_data_with_ID(conn, cursor, date_id_dict, "Date", date_key, date_data, batch_size)
            print("d")

            incident_id = int(row['incident_id'])
            incident_key = {"incident_id": incident_id}
            insert_data_with_ID(conn, cursor, incident_id_dict, "Incident", incident_key, incident_data, batch_size)
            print("e")

            #custody_id = row['custody_id']
            try:
        # ... (previous code)

                custody_id = row['custody_id']

                # Check if all required IDs are present
                if (
                    partecipant_id_dict and 
                    gun_id_dict and 
                    geo_id_dict and 
                    custody_id in partecipant_id_dict and 
                    custody_id in gun_id_dict and 
                    custody_id in geo_id_dict
                ):
                    partecipant_id = partecipant_id_dict[custody_id]
                    gun_id = gun_id_dict[custody_id]
                    geo_id = geo_id_dict[custody_id]

                    custody_key = {
                        "custody_id": custody_id,
                        "partecipant_id": partecipant_id,
                        "gun_id": gun_id,
                        "geo_id": geo_id,
                        "date_id": date_id,
                        "crime_gravity": compute_crime_gravity(row),
                        "incident_id": incident_id
                    }
                    insert_data_with_ID(conn, cursor, custody_id_dict, 'Custody', custody_key, custody_data, batch_size)
                    print("f")

                row_count += 1

                if row_count % batch_size == 0:
                    gun_id_list.clear()
                    geo_id_list.clear()
                    partecipant_id_list.clear()
                    print(row_count)
                    conn.commit()

                rows_to_upload -= batch_size
                print("row_count", row_count)

            except Exception as e:
                print(f"Error processing row: {e}")
                print("Problematic row:", row)
                continue

            
               


In [13]:
# Call the function with the appropriate arguments
split_and_integrate('Police.csv')

# Close the database connection when done
#cursor.close()
#conn.close()

170928 10
started working
a
b
c
d
e
row_count 1
started working
a
b
c
d
e
row_count 2
started working
a
b
c
d
e
row_count 3
started working
a
b
c
d
e
row_count 4
started working
a
b
c
d
e
row_count 5
started working
a
b
c
d
e
row_count 6
started working
a
b
c
d
e
row_count 7
started working
a
b
c
d
e
row_count 8
started working
a
b
c
d
e
row_count 9
started working
a
b
c
d
e
10
row_count 10
started working
a
b
c
d
e
row_count 11
started working
a
b
c
d
e
row_count 12
started working
a
b
c
d
e
row_count 13
started working
a
b
c
d
e
row_count 14
started working
a
b
c
d
e
row_count 15
started working
a
b
c
d
e
row_count 16
started working
a
b
c
d
e
row_count 17
started working
a
b
c
d
e
row_count 18
started working
a
b
c
d
e
row_count 19
started working
a
b
c
d
e
20
row_count 20
started working
a
b
c
d
e
row_count 21
started working
a
b
c
d
e
row_count 22
started working
a
b
c
d
e
row_count 23
started working
a
b
c
d
e
row_count 24
started working
a
b
c
d
e
row_count 25
started working
a


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=41.2226&lon=-80.7978&format=json&accept-language=en&addressdetails=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001548D14C110>: Failed to establish a new connection: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions'))

In [ ]:
no

In [ ]:
def insert_data_with_ID(conn, cursor, id_dict, table_name, key_dict, data_list, batch_size):
    key_tuple = tuple(key_dict.values())
    if key_tuple not in id_dict:
        id_dict[key_tuple] = next(iter(key_dict.values()))
        columns = ', '.join(key_dict.keys())
        placeholders = ', '.join(['?'] * len(key_dict))
        insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
        data_list.append(list(key_dict.values()))

        if len(data_list) >= batch_size:
            cursor.executemany(insert_query, data_list)
            conn.commit()
            data_list.clear()

def insert_data_without_ID(conn, cursor, table_name, key_dict, data_list, id_list, batch_size):
    columns = ', '.join(key_dict.keys())
    placeholders = ', '.join(['?'] * len(key_dict))
    insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    select_query = f'SELECT SCOPE_IDENTITY();'

    data_list.append(list(key_dict.values()))
    cursor.execute(select_query)
    id_list.append(cursor.fetchone()[0])

    if len(data_list) >= batch_size:
        cursor.executemany(insert_query, data_list)
        conn.commit()
        data_list.clear()

def split_and_integrate(csv_file):
    geo_id_dict = {}
    gun_id_dict = {}
    partecipant_id_dict = {}
    date_id_dict = {}
    incident_id_dict = {}
    custody_id_dict = {}
    # Parse dates.xml to get date mapping
    date_mapping = parse_dates_xml('dates.xml')

    # List of table names in your database
    table_names = ['Custody', 'Geography', 'Gun', 'Date', 'Incident', 'Partecipant']

    # Delete existing records in the tables
    for table_name in table_names:
        cursor.execute(f'DELETE FROM {table_name}')
        conn.commit()

    run_size = 10
    # Read and process Police.csv
    with open(csv_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        rows_to_upload = 170928 #sum(1 for row in reader)
        #next(reader)  # Skip the header row

        
        # Set the batch size
        if(rows_to_upload >= run_size):
            batch_size = run_size
        else:
            batch_size = rows_to_upload
        print(rows_to_upload, batch_size)
        # Counter to track the number of processed rows
        row_count = 0

        gun_data, participant_data, geo_data, date_data, incident_data, custody_data = [], [], [], [], [], []
        gun_id_list, geo_id_list, partecipant_id_list = [], [], []

        for row in reader:
            print("not working")
            custody_id, participant_age_group, participant_gender, participant_status, participant_type, latitude, longitude, gun_stolen, gun_type, incident_id, date_fk = row

            gun_stolen_bool = 1 if row["gun_stolen"] == 'Stolen' else 0
            gun_key_dict = {"is_stolen": gun_stolen_bool, "gun_type": row['gun_type']}
            insert_data_without_ID(conn, cursor, 'Gun', gun_key_dict, gun_data, gun_id_list, batch_size)
            print("a")

            partecipant_key = {
                "age_group" : row['participant_age_group'], 
                "gender" : row['participant_gender'], 
                "type" : row['participant_type'], 
                "status" : row['participant_status']
            }
            insert_data_without_ID(conn, cursor, 'Partecipant', partecipant_key, participant_data, partecipant_id_list, batch_size)
            print("b")
            # Get location information from latitude and longitude
            latitude, longitude = float(row['latitude']), float(row['longitude'])
            location_info = get_location_info(latitude, longitude)
            city = location_info["city"]
            state = location_info["state"]
            country = "United States"
            continent = "North America"
            geo_key = {
                "latitude" : str(latitude), "longitude" : str(longitude), "city" : city, "state" : state, "country" : country, "continent" : continent
            }
            insert_data_without_ID(conn, cursor, 'Geography', geo_key, geo_data, geo_id_list, batch_size)
            print("c")

            # Normal ID, No Incremental Tables:
            date_id = int(row['date_fk'])
            date_value = date_mapping[date_id]
            date, day, month, year, quarter, day_of_week = compute_date_data(date_value)
            date_key ={
                "date_id": date_id, "the_date" : date, "the_day" : day, "the_month" : month, "the_year" : year, "quarter" : quarter, "day_of_week" : day_of_week
            }
            insert_data_with_ID(conn, cursor, date_id_dict, "Date", date_key, date_data, batch_size)
            print("d")

            incident_id = int(row['incident_id'])
            incident_key = {"incident_id" : incident_id}
            insert_data_with_ID(conn, cursor, incident_id_dict, "Incident", incident_key, incident_data, batch_size)
            print("e")

            custody_id = row['custody_id']
            # Iterate through the lists
            for partecipant_id, gun_id, geo_id in zip(partecipant_id_list, gun_id_list, geo_id_list):
                # Create custody_key dictionary for each set of values
                custody_key = {
                    "custody_id": custody_id,
                    "partecipant_id": partecipant_id,
                    "gun_id": gun_id,
                    "geo_id": geo_id,
                    "date_id": date_id,
                    "crime_gravity": compute_crime_gravity(row),
                    "incident_id": incident_id
                    }
            insert_data_with_ID(conn, cursor, custody_id_dict, 'Custody', custody_key, custody_data, batch_size)
            print("f")

            row_count += 1
                # Commit in batches of 1000
            if row_count % batch_size == 0:
                gun_id_list.clear()
                geo_id_list.clear()
                partecipant_id_list.clear()
                print(row_count)
                conn.commit()    
            # Increment the row count
            

            rows_to_upload -= batch_size
            print("row_count", row_count)
    
                
    print(row_count)
    if row_count % batch_size != 0:
        conn.commit()


In [ ]:
custody_id = row['custody_id']
            # Iterate through the lists
            for partecipant_id, gun_id, geo_id in zip(partecipant_id_list, gun_id_list, geo_id_list):
                # Create custody_key dictionary for each set of values
                custody_key = {
                    "custody_id": custody_id,
                    "partecipant_id": partecipant_id,
                    "gun_id": gun_id,
                    "geo_id": geo_id,
                    "date_id": date_id,
                    "crime_gravity": compute_crime_gravity(row),
                    "incident_id": incident_id
                    }
            insert_data_with_ID(conn, cursor, custody_id_dict, 'Custody', custody_key, custody_data, batch_size)
            print("f")

IndentationError: unexpected indent (3708405867.py, line 3)

In [ ]:
# Call the function with the appropriate arguments
split_and_integrate('Police.csv')

# Close the database connection when done
#cursor.close()
#conn.close()

170928 10
not working
a
b
c
d
e
row_count 1
not working
a
b
c
d
e
row_count 2
not working
a
b
c
d
e
row_count 3
not working
a
b
c
d
e
row_count 4
not working
a
b
c
d
e
row_count 5
not working
a
b
c
d
e
row_count 6
not working
a
b
c
d
e
row_count 7
not working
a
b
c
d
e
row_count 8
not working
a
b
c
d
e
row_count 9
not working
a
b
c
d
e
10
row_count 10
not working
a
b
c
d
e
row_count 11
not working
a
b
c
d
e
row_count 12
not working
a
b
c
d
e
row_count 13
not working
a
b
c
d
e
row_count 14
not working
a
b
c
d
e
row_count 15
not working
a
b
c
d
e
row_count 16
not working
a
b
c
d
e
row_count 17
not working
a
b
c
d
e
row_count 18
not working
a
b
c
d
e
row_count 19
not working
a
b
c
d
e
20
row_count 20
not working
a
b
c
d
e
row_count 21
not working
a
b
c
d
e
row_count 22
not working
a
b
c
d
e
row_count 23
not working
a
b
c
d
e
row_count 24
not working
a
b
c
d
e
row_count 25
not working
a
b
c
d
e
row_count 26
not working
a
b
c
d
e
row_count 27
not working
a
b
c
d
e
row_count 28
not working
a

KeyboardInterrupt: 